In [ ]:
import pandas as pd

df = pd.read_csv("clean_df.csv")

#Esta es una de las primeras versiones de la función para calcular la disonancia ponderada y el ranking promedio
#Aquí tomo dos canciones de referencia y calculo las diferencias ponderadas con respecto a todas las demás canciones del ds
# Luego simplemente hay que unir los rankings de las dos canciones de referencia y ordenar por el ranking promedio
# Función para calcular la disonancia ponderada
#Para tampoco enrollarme mucho, básicamente, para cada característica (columna) calculo la diferencia entre el valor de referencia
#y el valor de la cancióna evaluar. Esas diferencias se multiplican por un peso que se le asigna a cada característica y se suman
#Ese valor es la disonancia total
#Se hace esto para cada canción y luego se ordena el dataset por disonancia total

def calcular_disonancia(row, referencia, importancias):
    disonancia_total = 0
    for columna, valor_referencia in referencia.items():
        importancia = importancias.get(columna, 1)
        diferencia = abs(row[columna] - valor_referencia)
        disonancia_total += diferencia * importancia
    return disonancia_total

# Función para obtener rankings de disonancia
def obtener_ranking(df, referencia, importancias, nombre_columna_ranking):
    df[nombre_columna_ranking] = df.apply(lambda row: calcular_disonancia(row, referencia, importancias), axis=1)
    df.sort_values(by=nombre_columna_ranking, ascending=True, inplace=True)
    df[nombre_columna_ranking + '_rank'] = range(1, len(df) + 1)
    return df

# Función principal para encontrar la canción con mejor ranking promedio
def encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista):
    # Obtener referencias de las dos canciones que pongamos
    referencia_a = df.loc[df[nombre_columna_cancion] == cancion_a].iloc[0]
    referencia_b = df.loc[df[nombre_columna_cancion] == cancion_b].iloc[0]

    # Extraer solo las características numéricas relevantes
    adn_a = referencia_a[importancias.keys()].to_dict()
    adn_b = referencia_b[importancias.keys()].to_dict()

    # Obtener rankings para cada referencia
    df = obtener_ranking(df, adn_a, importancias, "disonancia_a")
    df = obtener_ranking(df, adn_b, importancias, "disonancia_b")

    # Calcular el ranking promedio
    df["ranking_promedio"] = (df["disonancia_a_rank"] + df["disonancia_b_rank"]) / 2

    # Ordenar por ranking promedio
    columnas_a_mostrar = [nombre_columna_cancion, nombre_columna_artista, "disonancia_a_rank", "disonancia_b_rank", "ranking_promedio"]
    resultado = df[columnas_a_mostrar].sort_values(by="ranking_promedio")

    return resultado

# Diccionario de importancias
importancias = {
    "male": 1.0,
    "danceable": 0.9,
    "tonal": 0.05,
    "timbre_bright": 0.05,
    "instrumental": 0.9,
    "mood_acoustic": 0.7,
    "mood_aggressive": 1.0,
    "mood_electronic": 0.95,
    "mood_happy": 0.05,
    "mood_party": 1.0,
    "mood_relaxed": 1.0,
    "mood_sad": 1.0

}

# Especificar las dos canciones de referencia
cancion_a = "without me"  
cancion_b = "numb"


# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Encontrar la canción con mejor ranking promedio
df_resultado = encontrar_mejor_ranking_promedio(df, cancion_a, cancion_b, importancias, nombre_columna_cancion, nombre_columna_artista)

# Mostrar los resultados
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

print(df_resultado)

In [ ]:
import pandas as pd


df = pd.read_csv("completed_cleaned.csv")

#Este es el que seguramente acabaremos usando. Te devuelve las canciones más cercanas según los parámetros que le indiques y según su relevancia
#En su momento lo puse para el top 5 (y además te incluye la propia canción que le das como referencia). Se puede cambiar para que no saque la primera
#Y también para que saque la peor de todas, la que sería la antítesis de la canción original

# Función para calcular la disonancia ponderada
def calcular_disonancia(row, referencia, importancias):
    disonancia_total = 0
    for columna, valor_referencia in referencia.items():
        importancia = importancias.get(columna, 1)
        diferencia = abs(row[columna] - valor_referencia)
        disonancia_total += diferencia * importancia
    return disonancia_total

# Función para obtener rankings de disonancia
def obtener_ranking(df, referencia, importancias, nombre_columna_ranking):
    df[nombre_columna_ranking] = df.apply(lambda row: calcular_disonancia(row, referencia, importancias), axis=1)
    df.sort_values(by=nombre_columna_ranking, ascending=True, inplace=True)
    df[nombre_columna_ranking + '_rank'] = range(1, len(df) + 1)
    return df

# Función para obtener el top 5 de canciones menos disonantes para cada referencia
def obtener_top_5_por_referencia(df, canciones_referencia, importancias, nombre_columna_cancion, nombre_columna_artista):
    resultados_top_5 = {}

    for idx, cancion in enumerate(canciones_referencia, start=1):
        # Obtener la referencia de la canción actual
        referencia = df.loc[df[nombre_columna_cancion] == cancion].iloc[0]
        adn = referencia[importancias.keys()].to_dict()

        # Obtener ranking para la canción actual
        df_ranking = obtener_ranking(df.copy(), adn, importancias, f"disonancia_{idx}")

        # Seleccionar el top 5
        top_5 = df_ranking[[nombre_columna_cancion, nombre_columna_artista, f"disonancia_{idx}"]].head(5)
        resultados_top_5[cancion] = top_5

    return resultados_top_5

# Diccionario de importancias
importancias = {
    "male": 0.05,
    "danceable": 0.05,
    "tonal": 0.05,
    "timbre_bright": 0.05,
    "instrumental": 0.05,
    "mood_acoustic": 0.05,
    "mood_aggressive": 0.05,
    "mood_electronic": 0.05,
    "mood_happy": 0.05,
    "mood_party": 1.0,
    "mood_relaxed": 0.05,
    "mood_sad": 0.05
}

# Lista de canciones de referencia
canciones_referencia = ["ave maria", "satisfaction", "little blue"]  # Añade más canciones si lo deseas

# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Obtener el top 5 de canciones menos disonantes para cada referencia
resultados_top_5 = obtener_top_5_por_referencia(df, canciones_referencia, importancias, nombre_columna_cancion, nombre_columna_artista)

# Mostrar los resultados
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

for cancion, top_5 in resultados_top_5.items():
    print(f"\nTop 5 canciones menos disonantes para '{cancion}':")
    print(top_5)

In [ ]:
import pandas as pd

df = pd.read_csv("clean_df.csv")

# Este otro código es para calcular la canción "central". Para resolver el problema del arranque en frío.
#Lo que hace es comparar todas con todas (por pares de puntos) y calcular la diferencia entre ellas. Esto es a lo que me refería con "distancia"
#La idea es que las canciones que están a menor distancia de todas las demás es porque están más centralizadas

# Función para calcular la tasa de diferencia entre las canciones de referencia
def calcular_tasa_diferencia_referencias(df, referencias, columnas_parametros):
    diferencias = []
    canciones_referencia = []
    promedio_diferencias = {}
    
    # Extraer las canciones de referencia del dataframe
    for referencia in referencias:
        cancion_df = df[(df["song_name"] == referencia["cancion"]) & (df["artist_name"] == referencia["artista"])]
        if not cancion_df.empty:
            canciones_referencia.append(cancion_df.iloc[0])
            promedio_diferencias[referencia["cancion"]] = 0
    
    # Comparar todas las canciones de referencia entre sí
    for i in range(len(canciones_referencia)):
        for j in range(i + 1, len(canciones_referencia)):
            song_1 = canciones_referencia[i]
            song_2 = canciones_referencia[j]
            diferencia_total = 0
            
            for columna in columnas_parametros:
                valor_1 = song_1[columna] * 10
                valor_2 = song_2[columna] * 10
                diferencia_total += (abs(valor_1 - valor_2)) ** 2
            
            diferencias.append({
                "song_1": song_1["song_name"],
                "artist_1": song_1["artist_name"],
                "song_2": song_2["song_name"],
                "artist_2": song_2["artist_name"],
                "tasa_diferencia": diferencia_total
            })
            
            # Acumular diferencias para el promedio
            promedio_diferencias[song_1["song_name"]] += diferencia_total
            promedio_diferencias[song_2["song_name"]] += diferencia_total
    
    # Calcular el promedio de diferencia para cada canción
    for cancion in promedio_diferencias:
        promedio_diferencias[cancion] /= (len(canciones_referencia) - 1)
    
    # Convertir a DataFrame y ordenar
    diferencias_df = pd.DataFrame(diferencias)
    diferencias_df.sort_values(by="tasa_diferencia", ascending=True, inplace=True)
    
    # Crear DataFrame con promedios y ordenarlo
    promedio_df = pd.DataFrame(list(promedio_diferencias.items()), columns=["song_name", "promedio_diferencia"])
    promedio_df.sort_values(by="promedio_diferencia", ascending=True, inplace=True)
    
    return diferencias_df, promedio_df

# Diccionario de importancias
importancias = {
    "male": 0.5,
    "danceable": 1.2,
    "tonal": 0.8,
    "timbre_bright": 1.0,
    "instrumental": 1.5,
    "mood_acoustic": 0.7,
    "mood_aggressive": 1.3,
    "mood_electronic": 1.1,
    "mood_happy": 0.9,
    "mood_party": 1.4,
    "mood_relaxed": 0.6,
    "mood_sad": 1.0
}

# Lista de canciones de referencia con artista
referencias = [
    {"cancion": "mas guapa que cualquiera", "artista": "joaquin sabina"},
    {"cancion": "contrabando", "artista": "joaquin sabina"},
    {"cancion": "mi primo el nano", "artista": "joaquin sabina"},
    {"cancion": "while were young", "artista": "barry manilow"}
]

# Nombres de las columnas relevantes
nombre_columna_cancion = "song_name"
nombre_columna_artista = "artist_name"

# Obtener la tasa de diferencia entre las canciones de referencia
columnas_parametros = list(importancias.keys())
diferencias_df, promedio_df = calcular_tasa_diferencia_referencias(df, referencias, columnas_parametros)

# Mostrar los resultados
print("\nTasa de diferencia entre canciones de referencia:")
print(diferencias_df)

print("\nRanking de canciones por menor diferencia promedio:")
print(promedio_df)
